In [1]:
# packages to import, read in the latest dataframe pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import matplotlib
from colour import Color
from glob import glob
import re
import PIL
from PIL import Image

matplotlib.rcParams['pdf.fonttype'] = 42

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 14}

matplotlib.rc('font', **font)

pd.options.mode.chained_assignment = None 

In [2]:
#this makes a custom colormap for visualizing the footprints
#you input a list of colors (html codes), and a corresponding list of sizes
#this will generate a colormap which will have the given color at the corresponding size, and then will interpolate colors to the next size

def make_cp(colors, sizes):
    cp =[tuple([0,0,0,0])]*sizes[0] #0-20
    for i in range(len(colors)-1):
        cs=Color(colors[i])
        ce=Color(colors[i+1])
        for j in list(cs.range_to(ce, sizes[i+1]-sizes[i])):
            j=list(j.rgb)
            j.append(1)
            j=tuple(j)
            cp.append(j)

    return matplotlib.colors.ListedColormap(cp)

#the colormap I use in general

#colors Pol II and PIC sized footprints
colors=['#ed306f','#e42cf6','#f3aafa','#265de6','#33a040'] 
sizes=[20,            45,        55,   70,         100]
pol_cp=make_cp(colors, sizes)

In [3]:
#this makes a dataframe used to plot enrichment of specific footprint sizes w.r.t. position
#s is the start coordinate, e is the end coordinate
#min_s, max_s are the min and max size
#bin_size is if you want to do bins of sizes (so like every 5nt or whatever)

def plot_meta_heatmap(reads_df,s,e, min_s,max_s,bin_size):
    m_d=pd.DataFrame()
    reads_df=reads_df.fillna(0)[np.arange(s,e).astype(str)]
    reads_df[reads_df<=2]=0
    d=[]
    for s in range(min_s//bin_size,max_s//bin_size):
        df=reads_df.copy()
        df=df.where(df >= (s)*bin_size,0)
        df=df.where(df < (s+1)*bin_size,0)
        a=pd.DataFrame(df.sum(axis=0)).T
        d.append((s*bin_size))
        m_d=pd.concat([m_d,a], axis=0)
    m_d.index=d
    return m_d

In [4]:
#this generates an array with the enrichment of a given size range of footprints in a given range, normalized by read count at the position

def meta_line_plot(reads_df, start, end, fp_size_min, fp_size_max):
    
    df=reads_df.copy()
    
    count_df=df.copy()
    count_df[count_df!=2]=1
    count_df[count_df==2]=0
    counts=count_df.sum(axis=0)
    counts=counts.loc[np.arange(start,end).astype(str)]
    
    df[df <=2 ] = 0

    df=df.T
    df=df.loc[np.arange(start,end).astype(str)]
    df=df.T

    if fp_size_min:
        df=df.where(df > fp_size_min, 0)
    if fp_size_max:
        df=df.where(df < fp_size_max, 0)
    df=df.where(df == 0, 1)    
    
    df_a=df.sum(axis=0)
    df_a=np.array(df_a)/counts.to_numpy()
    df=''
    
    return df_a

In [5]:
# Define a function to read in the data
def read_data(file_paths, s, e):
    df_dict = {}
    for path in file_paths:
        # read in the path as a dataframe
        df = pd.read_csv(path, sep='\t', names=np.arange(s,e).astype(str))
        # extract df_name
        df_name = f'{path.split("/")[7]}_{path.split("/")[9].split("_")[0]}'
        # store the dataframe in the dictionary with the df_name as the key
        df_dict[df_name] = df
    return df_dict

In [6]:
#import the data
#you can import the entire dataset by leaving out the columns argument or you can choose a position range (s and e)
#parquet files require strings as their column names so you have to turn the arange into strings
#I would import at minimum 100 or so bases beyond the range you want to work with, just so that the smoothing for some of these plots doesn't have issues
# pEF-GFP
# s=0
# e=5054

# Define the base path to directory
base_path = '/gscratch/stergachislab/bmallo/large_home/hmm-footprint-caller/pEF_GFP_Gus/'

# Define the pattern to match the files
# pattern_WT = f'{base_path}Gus_WT*/Outfiles/*_nucleosomes.m6a.gz/*.tsv'
# pattern_Mut = f'{base_path}Gus_MUT*/Outfiles/*_nucleosomes.m6a.gz/*.tsv'
pattern_GFP = f'{base_path}pEF_GFP_rep3/Outfiles/*_nucleosomes.m6a.gz/*.tsv'

# Use glob to get a list of file paths
# WT_paths = glob(pattern_WT)
# Mut_paths = glob(pattern_Mut)
GFP_paths = glob(pattern_GFP)

# Create a dictionary to store dataframes in
# SLC39A4_WT = read_data(WT_paths, 0, 6462)
# SLC39A4_Mut = read_data(Mut_paths, 0, 6462)
pEF_GFP = read_data(GFP_paths, 0, 5054)



In [7]:
def filter_dataframes(df_dict, threshold):
    df_dict_filtered = {}
    for df_name, df in df_dict.items():
        # Filter the dataframe to remove all reads whose first footprint is greater than threshold
        filtered_df = df[(abs(df.iloc[:, 1]) <= threshold) & (abs(df.iloc[:, 0]) <= threshold)]

        # Store the filtered dataframe back in the dictionary
        df_dict_filtered[df_name] = filtered_df
    return df_dict

# Filter dataframes
# SLC39A4_WT_filtered = filter_dataframes(SLC39A4_WT, 1000)
# SLC39A4_Mut_filtered = filter_dataframes(SLC39A4_Mut, 1000)
pEF_GFP_filtered = filter_dataframes(pEF_GFP, 1000)


In [11]:
#plot the footprint size heatmap
bin_size=2
min_size=40
max_size=250
start=s
end=e

meta=plot_meta_heatmap(Gus_WT_Rep2_filterd, start, end, min_size, max_size, bin_size)

fig, ax=plt.subplots(1,1,figsize=(10,5))
sns.heatmap(data=meta, cbar=True, cmap='gray', ax=ax, xticklabels=1000, yticklabels=10)
ax.set_xlabel('genome position')
ax.set_ylabel('Size of Footprint (5bp bins)')
plt.title('Gus WT nuclear localized plasmid footprints')
plt.show()

NameError: name 's' is not defined

In [57]:
#plot the enrichment of PIC, pause, nucleosome sized footprints
s=25
e=5029
smooth=25
i=0

# Define samples to plot
# allele_1 = 'WT'
# allele_2 = 'MUT'    
count_1 = 35
# count_2 = 3
# replicate_1 = 'rep3'
# replicate_2 = 'rep3'
nuc_range = (90, 1000)
PIC_range = (60, 80)

# sample_1 = SLC39A4_WT_filtered[f'Gus_{allele_1}_{replicate_1}_{count_1}']
# sample_2 = SLC39A4_Mut_filtered[f'Gus_{allele_2}_{replicate_2}_{count_2}']
sample_3 = pEF_GFP_filtered[f'pEF_GFP_rep3_{count_1}']

fig, ax = plt.subplots(1,1,figsize=(10,5))

#grab the enrichment for the different size ranges (can adjust these as you need)
a=meta_line_plot(sample_3,  s-smooth, e+smooth, nuc_range[0], nuc_range[1])
# b=meta_line_plot(sample_2,  s-smooth, e+smooth, nuc_range[0], nuc_range[1])
c=meta_line_plot(sample_3,  s-smooth, e+smooth, PIC_range[0], PIC_range[1])
# d=meta_line_plot(sample_2,  s-smooth, e+smooth, PIC_range[0], PIC_range[1])

#do some smoothing to make it look pretty
box = np.ones(smooth)/smooth
a = np.convolve(a, box, mode='same')
a = a[smooth:len(a)-smooth]
# b = np.convolve(b, box, mode='same')
# b = b[smooth:len(b)-smooth]
c = np.convolve(c, box, mode='same')
c = c[smooth:len(c)-smooth]
# d = np.convolve(d, box, mode='same')
# d = d[smooth:len(d)-smooth]


# plot PIC/Pol II
ax.plot(np.arange(s,e),c, color='#009688', linestyle='solid', linewidth=1.5, label=f'PIC')
# ax.plot(np.arange(s,e),d, color='#F57C00', linestyle='solid', linewidth=1.5, label=f'{count_2}')
ax.set_ylim(0, 0.1)
ax.set_xlim(s,e)

# Add vertical line at SNP position
#vline = ax.axvline(x=6119, color='black', linestyle='dotted', linewidth=1.4, label='A>G')

#plot nucleosomes on a twin axis bc there are way more
ax2 = ax.twinx() 
#ax2.set_xlim(s,e)
#ax2.set_ylim(0, 1)
# ax2.plot(np.arange(s,e),a, color='#009688', label=f'{allele_1}: {count_1} nucs')
ax2.plot(np.arange(s,e),a, color='#F57C00', label=f'{count_1} nucs')
# ax2.plot(np.arange(s,e),b, color='#F57C00', label=f'{allele_2}: {count_2} nucs')

# Combine legend handles and labels
# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# lines2.append(lines[2]) # add the nucleosome lines to the legend
# labels2.append(labels[2])

# Add legend manually
ax.legend(loc='upper right', frameon=False, bbox_to_anchor=(1.05, 1.2), fontsize=10)
ax.set_ylim(0, 0.15)
ax2.set_ylim(0, 1)

# Set labels and title 
plt.xlabel('genomic position')
ax.set_ylabel('footprint enrichment (Pol II)')
ax2.set_ylabel('footprint enrichment (Nucleosomes)')
plt.title(f'pEF-GFP Rep3: {count_1}')
plt.legend(loc='upper right', frameon=False, bbox_to_anchor=(1.09, 1.15), fontsize=10)
plt.tight_layout()

# Save figure
output_path = f'/gscratch/stergachislab/bmallo/large_home/hmm-footprint-caller/figures/pEF-GFP_rep3/pEF-GFP_rep3_{count_1}_nucs.png'

# Add metadata
# Set standardized metadata information
metadata = {
    'Nucleosome Sizes': str(nuc_range),
    'PIC sizes': str(PIC_range),  # You can include custom keys as well
}

# Save figure
plt.savefig(output_path, metadata=metadata, bbox_inches='tight', dpi=300)


KeyError: 'pEF_GFP_rep3_35'

In [53]:
def plot_enrichment(s, e, smooth, allele_1, allele_2, count_1, count_2, replicate_1, replicate_2, nuc_range=(90,10000), PIC_range=(60, 80)):

    sample_1 = SLC39A4_WT_filtered[f'Gus_{allele_1}_{replicate_1}_{count_1}']
    sample_2 = SLC39A4_Mut_filtered[f'Gus_{allele_2}_{replicate_2}_{count_2}']

    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    a = meta_line_plot(sample_1, s - smooth, e + smooth, nuc_range[0], nuc_range[1])
    b = meta_line_plot(sample_2, s - smooth, e + smooth, nuc_range[0], nuc_range[1])
    c = meta_line_plot(sample_1, s - smooth, e + smooth, PIC_range[0], PIC_range[1])
    d = meta_line_plot(sample_2, s - smooth, e + smooth, PIC_range[0], PIC_range[1])

    box = np.ones(smooth) / smooth
    a = np.convolve(a, box, mode='same')[smooth:len(a) - smooth]
    b = np.convolve(b, box, mode='same')[smooth:len(b) - smooth]
    c = np.convolve(c, box, mode='same')[smooth:len(c) - smooth]
    d = np.convolve(d, box, mode='same')[smooth:len(d) - smooth]

    ax.plot(np.arange(s,e),c, color='#009688', linestyle='dashed', linewidth=1.5, label=f'{count_1}')
    ax.plot(np.arange(s,e),d, color='#F57C00', linestyle='dashed', linewidth=1.5, label=f'{count_2}')
    ax.set_ylim(0, 0.1)
    ax.set_xlim(s, e)

    vline = ax.axvline(x=6119, color='black', linestyle='dotted', linewidth=1.4, label='A>G')
    
    ax2 = ax.twinx()
    ax2.set_ylim(0, 1)
    ax2.plot(np.arange(s,e),a, color='#009688', label=f'{allele_1}: {count_1} nucs')
    ax2.plot(np.arange(s,e),b, color='#F57C00', label=f'{allele_2}: {count_2} nucs')

    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    lines2.append(lines[2])
    labels2.append(labels[2])

    ax.legend(lines2, labels2, loc='upper right', frameon=False, bbox_to_anchor=(1.05, 1.2), fontsize=10)

    plt.xlabel('genomic position')
    ax.set_ylabel('footprint enrichment (Pol II)')
    ax2.set_ylabel('footprint enrichment (Nucleosomes)')
    plt.title(f'SLC39A4: WT ({replicate_1}) vs MUT ({replicate_2})')
    plt.tight_layout()

    output_path = f'/gscratch/stergachislab/bmallo/large_home/hmm-footprint-caller/figures/SLC39A4_WT_vs_MUT_2/SLC39A4_WT_vs_MUT_{replicate_1}_vs_{replicate_2}_{count_1}_nucs.png'

    metadata = {
        'Nucleosome Sizes': str(nuc_range),
        'PIC sizes': str(PIC_range),
    }

    plt.savefig(output_path, metadata=metadata, bbox_inches='tight', dpi=300)
    plt.close(fig)

In [54]:
for count in range(1, 45):
    for replicate in ['rep1', 'rep2', 'rep3']:
        count_1 = count_2 = count
        replicate_1 = replicate_2 = replicate

        try:
            plot_enrichment(s=25, e=6437, smooth=25, allele_1='WT', allele_2='MUT',
                            count_1=count_1, count_2=count_2, replicate_1=replicate_1, replicate_2=replicate_2)
        except KeyError:
            # Handle the case where the sample does not exist
            print(f"Sample not found for count={count} and replicate={replicate}. Skipping...")


Sample not found for count=40 and replicate=rep1. Skipping...
Sample not found for count=41 and replicate=rep1. Skipping...
Sample not found for count=42 and replicate=rep1. Skipping...
Sample not found for count=42 and replicate=rep2. Skipping...
Sample not found for count=43 and replicate=rep1. Skipping...
Sample not found for count=43 and replicate=rep2. Skipping...
Sample not found for count=43 and replicate=rep3. Skipping...
Sample not found for count=44 and replicate=rep1. Skipping...
Sample not found for count=44 and replicate=rep2. Skipping...
Sample not found for count=44 and replicate=rep3. Skipping...


In [ ]:
#plot all reads in a given region, colored by footprint identity

s=25
e=6400

fig, ax=plt.subplots(1,1, figsize=(10,30))

d=SLC39A4_WT_filtered['Gus_WT_rep3_22']
#you can do stuff like k-means clustering the dataframe or sorting by the distance to the 1st nucleosome or something to get different orders of reads if you want

# Specify the number of rows and the column index to center on
num_rows_to_plot = 400
center_column_index = 6000  # Replace with the desired column index

#these plot the white space and black lines
for i in range(d.shape[0]+1):
    ax.axhline(i+.5, color='black', lw=.25, zorder=1)
for i in range(d.shape[0]+1):
    ax.axhline(i, color='white', lw=2, zorder=3)

#this plots the actual reads
sns.heatmap(data=d,
cmap=pol_cp, vmin=0, vmax=1000,
            yticklabels=False, cbar=False, ax=ax,
            xticklabels=500)
plt.tight_layout()
plt.show()